## Data

In [1]:
language_words = {}

In [2]:
import unicodedata
import string

def process_word(word):
    new_str = ''
    
    # Removes accents, digits, puntuation and other stuff, 
    # and keeps characters only
    for char in unicodedata.normalize("NFD", word):
        if unicodedata.category(char) != "Mn" and char.isalpha():
            new_str += char
            
    return new_str

In [3]:
def collect_words(txt):
    with open(txt) as f:
        txt_data = f.read()
        
    # Places words into a list by splitting space.
    words = txt_data.split()
    # Only keeps characters.
    words = [process_word(word) for word in words]
    # Removes duplicates.
    words = list(set(words))
    
    return words

In [4]:
import glob
import os

for txt in glob.glob("data_/*.txt"):
    language = os.path.splitext(os.path.basename(txt))[0]
    words = collect_words(txt)
    
    language_words[language] = words

print(language_words)

{'German': ['', 'dienen', 'ServicesOutsourcing', 'Linguistik', 'Universidad', 'acht', 'YunshanGelehrter', 'belauft', 'Guangzhou', 'Berufung', 'renommierten', 'lasst', 'Lehrkrafte', 'mehr', 'Außerdem', 'durch', 'Prufungsgremiums', 'Kolleg', 'Taiwan', 'Standorte', 'Zusammenarbeit', 'als', 'gesinnte', 'Millionen', 'schafft', 'Kunste', 'ist', 'eines', 'verschiedenen', 'Talente', 'auf', 'Hundert', 'Forschungszentren', 'Kreativitat', 'Hektar', 'Schwerpunktforschungszentrum', 'ausgezeichneten', 'Ausbildungsmodelle', 'Mitglied', 'Mexiko', 'CampusNetwork', 'Befugnis', 'dem', 'Netzwerk', 'Innovationen', 'Kollegium', 'sollten', 'durchgefuhrt', 'wurde', 'Wirtschaft', 'hervor', 'Japan', 'Fachschule', 'Asien', 'Studienfacher', 'auslandische', 'wettbewerbsfahigen', 'Anlagevermogen', 'Ebene', 'DalangCampus', 'Schwerpunktdisziplinen', 'Naturwissenschaft', 'Padagogik', 'Luft', 'Quadratmetern', 'PerlflussGelehrter', 'inlandische', 'Professoren', 'Spezialisten', 'Ausbildung', 'den', 'Maria', 'Nationen', '